# Grundlegende Infos

**Aufgabe:**
Zusammenführen der Ergebnis-Dateien

**Eingabedateien:** XLSX-Dateien

# Notwendige Imports & Vorbereitung
## Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import os
import sqlite3

## Anlegen der SQLite Datenbank

In [2]:
conn = sqlite3.connect("../working/db.sqlite")

stmt = """
create table athletes
(
    id         integer
        constraint athletes_pk
            primary key autoincrement,
    name       TEXT,
    first_name TEXT,
    gender     TEXT,
    year       TEXT,
    club       TEXT,
    category   TEXT
);
"""

conn.execute(stmt)

# Einstellungen
## Verzeichnisse

In [3]:
input_dir = "../result"
output_dir = "../finished"

# Notwendige Funktionen

**ID des Athleten**

In [21]:
def get_athlete_id (name, first_name, year):
    stmt = """
        SELECT id from athletes WHERE name=? AND first_name=? AND year=?
    """
    c = conn.cursor()
    result = c.execute(stmt, (name, first_name, year))
    id = c.fetchone()
    c.close()
    return id[0]

**Einfügen eines Athleten**

In [13]:
def insert_athlete (name, first_name, year, gender, club, category):
    # Check if athlete exists
    if get_athlete_id(name, first_name, year) is not None:
        return

    # Insert Athlete
    c = conn.cursor()

    stmt = """
    insert into athletes (name, first_name, gender, "year", club, category) values (?, ?, ?, ?, ?, ?);
    """

    c.execute(stmt, (name, first_name, gender, year, club, category))
    conn.commit()
    c.close()

# Einlesen der Athleten

In [17]:
row = []

for file in os.listdir(input_dir):
    # Ignore open files
    if file.startswith("~"):
        continue

    file_path = os.path.join(input_dir, file)

    for gender in ("male", "female"):
        df_excel = pd.read_excel(file_path, gender, index_col=0)

        for index, row in df_excel.iterrows():
            insert_athlete(row["Nachname"], row["Vorname"], row["Geb. Jahr"], gender, row["Verein"], row["Altersklasse"])

conn.commit()

# Einlesen der Ergebnisse

In [24]:
cursor = conn.cursor()
for file in os.listdir(input_dir):
    # Ignore open files
    if file.startswith("~"):
        continue

    file_path = os.path.join(input_dir, file)

    # Get table name
    table_name = file.split(".",1)[0]

    # Create the table
    stmt = """
    create table if not exists %s
    (
        athlete_id integer,
        sum        integer
    );
    """
    conn.execute(stmt % table_name)

    insert_stmt = """
    insert into %s (athlete_id, "sum")
    values (?, ?);
    """

    # Add the points
    for gender in ("male", "female"):
        df_excel = pd.read_excel(file_path, gender, index_col=0)

        for index, row in df_excel.iterrows():
            athlete_id = get_athlete_id(row["Nachname"], row["Vorname"], row["Geb. Jahr"])

            if athlete_id is None:
                raise Exception("No athlete found")

            cursor.execute(insert_stmt % table_name, (athlete_id, row["Summe"]))


conn.commit()
cursor.close()

# Ausgeben der Ergebnisse

In [27]:
result_stmt = """
SELECT
    a.name,
    a.first_name,
    a.gender,
    a.year,
    a.club,
    a.category,
    s.sum AS "SM",
    l.sum AS "LJS"
FROM
    athletes AS a
LEFT JOIN sm s on a.id = s.athlete_id
LEFT JOIN ljs l on a.id = l.athlete_id
WHERE category is NOT NULL
"""

In [30]:
def calc_points_per_athlete(row):
    # Remove Name columns
    row = row.tail(-6)
    # Convert to numbers
    row = pd.to_numeric(row)
    # Get the 3 largest numbers
    row = row.nlargest(3)
    # Create the sum
    sum = row.sum()

    return sum

In [32]:
df_result = pd.read_sql_query(result_stmt, conn)
df_result.fillna(0, inplace=True)
df_result["Summe"] = df_result.apply(calc_points_per_athlete, axis=1)
df_result.sort_values(by=["gender","category", "Summe"], ascending=[True, True, False], inplace=True)
df_result

,name,first_name,gender,year,club,category,SM,LJS,Summe
33,Ullrich,Johanna Hermine,female,2007,1. Chemnitzer Tauchverein e.V.,2006 & 2007,146.0,146.0,292.0
34,Marquardt,Lotte,female,2007,SG Dresden,2006 & 2007,142.0,146.0,288.0
37,Bretschneider,Luisa,female,2007,SG Dresden,2006 & 2007,114.0,124.0,238.0
35,Neumann,Josephine,female,2006,SG Dresden,2006 & 2007,138.0,84.0,222.0
32,Holtz,Leonie-Florentine,female,2007,SC DHfK Leipzig Flossenschwimmen,2006 & 2007,150.0,0.0,150.0
...,...,...,...,...,...,...,...,...,...
81,Schewitzer,Gustav,male,2013,Startgemeinschaft Dresden,2012 & 2013,0.0,85.0,85.0
29,Michalke,Max-Leon,male,2014,Tauchclub NEMO Plauen e.V.,2014 & jünger,150.0,150.0,300.0
31,Fleck,Maximilian,male,2014,SC Riesa Sekt. Flossenschwimmen,2014 & jünger,84.0,126.0,210.0
82,vom Hoff,Neo,male,2014,SC Riesa (TC),2014 & jünger,0.0,138.0,138.0


In [33]:
finished_file = os.path.join(output_dir, "results.xlsx")
df_result.to_excel(finished_file)